In [1]:
#!git clone https://github.com/remaplab/RecSys_Course_AT_PoliMi.git

Cloning into 'RecSys_Course_AT_PoliMi'...


remote: Enumerating objects: 1672, done.


remote: Counting objects: 100% (244/244), done.
remote: Compressing objects: 100% (35/35), done.


remote: Total 1672 (delta 221), reused 209 (delta 209), pack-reused 1428 (from 3)
Receiving objects: 100% (1672/1672), 53.49 MiB | 22.49 MiB/s, done.


Resolving deltas: 100% (972/972), done.


In [2]:
#rm /kaggle/working/RecSys_Course_AT_PoliMi/Recommenders/MatrixFactorization/PureSVDRecommender.py

In [3]:
#cp -a /kaggle/input/recsys-repo-resources/PureSVDRecommender.py /kaggle/working/RecSys_Course_AT_PoliMi/Recommenders/MatrixFactorization/

In [4]:
#cd /kaggle/working/RecSys_Course_AT_PoliMi/

/kaggle/working/RecSys_Course_AT_PoliMi


In [ ]:
import json
import pandas as pd
import scipy as sp
import numpy as np
np.int = np.int32
np.float = np.float64
np.bool = np.bool_



In [8]:
from src.Evaluation.Evaluator import EvaluatorHoldout
from fairness_serendipity_fns import compute_item_popularity, compute_serendipity, compute_fairness, compute_item_popularity,compute_recommendations_dict


urm_test=sp.sparse.load_npz("/splitted_urms/urm_test.npz")
urm_train=sp.sparse.load_npz("/splitted_urms/urm_train.npz")
urm_validation=sp.sparse.load_npz("/splitted_urms/urm_validation.npz")

evaluator = EvaluatorHoldout(urm_validation, cutoff_list=[10])
item_popularity=compute_item_popularity(urm_train)

In [ ]:
with open('config_utils.json','r') as f:
    data=json.load(f)
    configurations_dict=data['weights_config']
    num_configs=data['num_configs']

In [ ]:
from src.Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDItemRecommender


def objective_function_pure_svd(trial):
    params = {
        'topK': trial.suggest_int('topK', 1, 1000, log=True),#1000
        'num_factors': trial.suggest_int('num_factors', 1, 1000, log=True) #1000
    }

    recommender_instance = PureSVDItemRecommender(urm_train)
    recommender_instance.fit(**params,random_seed=42)
    result_df, _ = evaluator.evaluateRecommender(recommender_instance)
    return result_df.loc[10, "HIT_RATE"]


def multi_objective_function_pure_svd(trial):
    params = {
        'topK': trial.suggest_int('topK', 1, 1000, log=True),#1000
        'num_factors': trial.suggest_int('num_factors', 1, 1000, log=True) #1000
    }

    recommender_instance = PureSVDItemRecommender(urm_train)
    recommender_instance.fit(**params,random_seed=42)

    rec_dict=compute_recommendations_dict(recommender_instance, urm_validation)
    result_df, _ = evaluator.evaluateRecommender(recommender_instance)
    fairness=compute_fairness(rec_dict, item_popularity, G=4)
    serendipity=compute_serendipity(rec_dict, item_popularity)
    composed_metrics=weights_list[0]*result_df.loc[10, "HIT_RATE"] +  weights_list[1]*result_df.loc[10,"DIVERSITY_GINI"] +weights_list[2]*result_df.loc[10, "NOVELTY"]+ weights_list[3]*serendipity+weights_list[4]*fairness
    return composed_metrics

In [9]:
import optuna
from optuna.samplers import TPESampler

for i in range(1,num_configs+1):
    config=f"c{i}"
    weights_list=configurations_dict[config]
    database_path=f"/kaggle/working/pureSVD_multi_{config}.db"
    sampler=TPESampler(n_startup_trials=6)
    study=optuna.create_study(study_name=f"pureSVD_multi_hyper_param_tuning_{config}",sampler=sampler,direction="maximize",storage=f'sqlite:///{database_path}',load_if_exists=True)
    
    study.optimize(multi_objective_function_pure_svd, n_trials=50)


[I 2025-11-16 13:34:02,952] A new study created in RDB with name: pureSVD_multi_hyper_param_tuning_c1


PureSVDItemRecommender: Computing SVD decomposition...


PureSVDItemRecommender: Computing SVD decomposition... Done!


EvaluatorHoldout: Processed 6022 (100.0%) in 2.44 sec. Users per second: 2464


[I 2025-11-16 13:34:07,838] Trial 0 finished with value: 0.2645999583374259 and parameters: {'topK': 73, 'num_factors': 1}. Best is trial 0 with value: 0.2645999583374259.


...

EvaluatorHoldout: Processed 6022 (100.0%) in 2.59 sec. Users per second: 2323


[I 2025-11-16 13:44:33,127] Trial 45 finished with value: 0.5670239030722637 and parameters: {'topK': 108, 'num_factors': 30}. Best is trial 32 with value: 0.5706312409312245.


PureSVDItemRecommender: Computing SVD decomposition...


PureSVDItemRecommender: Computing SVD decomposition... Done!


EvaluatorHoldout: Processed 6022 (100.0%) in 2.85 sec. Users per second: 2113
